In [1]:
#import dependencies
import pandas as pd

In [3]:
#create csv path
stats = "InvoiceStats.csv"

# Read the CSV into a Pandas DataFrame
inv_stats = pd.read_csv(stats)

inv_stats.head()

,Billing Group,Invoice Status,Servicing Dealer,Dealer Invoice,Requested Amount,Approved Amount,Invoice Create Date,Invoice Issue?
0,ARM,Raymond Paid,60790,V10312509,105.00,105.00,02-12-2021,No
1,ARM,Raymond Paid,60790,V10312513,95.00,95.00,02-12-2021,No
2,ARM,Raymond Paid,60790,V10312515,105.00,105.00,02-12-2021,No
3,ARM,Raymond Paid,60789,V10312527,717.43,717.43,02-12-2021,No
4,ARM,Raymond Paid,60790,V10313561,119.85,119.85,02-24-2021,No


In [5]:
#Reorganize columns
invstats_df = inv_stats[["Servicing Dealer", "Billing Group", "Invoice Status", "Requested Amount", "Approved Amount", "Invoice Create Date", "Invoice Issue?"]]

invstats_df.head()

,Servicing Dealer,Billing Group,Invoice Status,Requested Amount,Approved Amount,Invoice Create Date,Invoice Issue?
0,60790,ARM,Raymond Paid,105.00,105.00,02-12-2021,No
1,60790,ARM,Raymond Paid,95.00,95.00,02-12-2021,No
2,60790,ARM,Raymond Paid,105.00,105.00,02-12-2021,No
3,60789,ARM,Raymond Paid,717.43,717.43,02-12-2021,No
4,60790,ARM,Raymond Paid,119.85,119.85,02-24-2021,No
